In [ ]:
import os
import glob
import copy
import numpy as np
import Bio
import scipy.spatial
import pickle
import matplotlib.pyplot as plt
import pandas as pd
from rnai_scripts import *
import bokeh.io
import bokeh.plotting

# Enable viewing Bokeh plots in the notebook
bokeh.io.output_notebook()

In [ ]:
def ecdf_vals(data):
    """Return x and y values for an ECDF."""
    return np.sort(data), np.arange(1, len(data)+1) / len(data)


# RNAi recoding 

## Reading in the Smed transcriptome
We read in the Smed_v6 transcriptome orfs that were extracted using orfipy. We then join them all into one string and obtain the codon frequencies. 

In [ ]:
fname = 'data/dd_Smed_v6_transcripts_orfs_large3.fa' # makes smallest proteins be around 30 amino acids
descriptors, seqs = read_many_fasta(fname)
# join all ORFS into one large transcriptome 
transcriptome = ''.join(seqs)
# get aminoacidweights and codon weights 

codon_frequencies_dic = get_codon_frequencies(transcriptome) 

Now we get frequencies of doublets

In [ ]:
doubletscode = get_codon_frequencies_doublets(transcriptome)

I also found a published version of amino acid frequencies:

In [ ]:
df = pd.read_csv('data/codon_usage_smed.csv')


AAs = df['codon'].values
freqs = df['frequency'].values/1000.

codon_frequencies_dic_published = {}
for i in range(len(AAs)):
    codon_frequencies_dic_published[AAs[i]] = freqs[i]
print(sum(freqs))

Let's calculate the average discrepency between the doublets vs. codon frequencies. 

In [ ]:
diff_published_vs_me = {}
for a in AAs:
    
    diff_published_vs_me[a] = codon_frequencies_dic_published[a] - codon_frequencies_dic[a]
values = np.array(list(diff_published_vs_me.values()))
print(np.mean(values))
print(np.mean(np.abs(values))) # values usually on order 
print(np.sum(np.abs(values)))

Here we find the discrepencies between the frequencies of each doublet vs. the product frequency of the separate codons. 

In [ ]:

diff_dic = {}
diff_dic_norm = {}
for pair in doubletscode.keys():
    if 'TAA' == pair[:3]:
        continue
    if 'TAG' == pair[:3]:
        continue
    if 'TGA' == pair[:3]:
        continue
    
    freq1 = codon_frequencies_dic[pair[:3]]
    freq2 = codon_frequencies_dic[pair[3:]]
    
    diff_dic_norm[pair] = (doubletscode[pair] - freq1*freq2)/np.max(np.array([freq1, freq2]))
    diff_dic[pair] = (doubletscode[pair] - freq1*freq2)

In [ ]:
# Make figure
p = bokeh.plotting.figure(
    frame_width=400,
    frame_height=300,
    x_axis_label='diff',
    y_axis_label='Dist',
  #  x_axis_type = 'log'
    
)
diffs, ecdf_diffs = ecdf_vals(np.array(list(diff_dic.values())))
print(np.sum(np.array(list(doubletscode.values()))))
p.circle(diffs*1e4, ecdf_diffs)

#diffs, ecdf_diffs = ecdf_vals(np.array(list(doublets.values())))
#p.circle(diffs, ecdf_diffs, color = 'orange')
bokeh.io.show(p)

In [ ]:
# Make figure
p = bokeh.plotting.figure(
    frame_width=400,
    frame_height=300,
    x_axis_label='diff',
    y_axis_label='Dist',
  #  x_axis_type = 'log'
    
)
diffs, ecdf_diffs = ecdf_vals(np.array(list(diff_dic_norm.values())))
print(np.sum(np.array(list(doubletscode.values()))))
p.circle(diffs, ecdf_diffs)

#diffs, ecdf_diffs = ecdf_vals(np.array(list(doublets.values())))
#p.circle(diffs, ecdf_diffs, color = 'orange')
bokeh.io.show(p)

In [ ]:
values = np.array(list(diff_dic_norm.values()))
inds_sort = np.argsort(values)
keys = np.array(list(diff_dic_norm.keys()))
keys[inds_sort][:100]

In [ ]:
values = np.array(list(diff_dic.values()))*1e4
inds_sort = np.argsort(values)
keys = np.array(list(diff_dic.keys()))
keys[inds_sort][:100]

In [ ]:
diff_dic['AAAAAA']*1e4

In [ ]:
doubletscode['AAAAAA']

In [ ]:
codon_frequencies_dic['AAA']*codon_frequencies_dic['AAA']

We use our codon frequencies dictionary to compute CAI weights (based on the weight definition for the CAI) for all codons 

$$w_i = \frac{f_i}{\max (f_j)} i,j \in [ \text{synonymouse codons for amino acid} ]$$

Where $f_i$ is the frequency of codon $i$. 

We obtain two dictionaries: 


aminoacidweights: keys are amino acids, values are arrays of $w_i$ for all synonymous codons. The order of the codons is the as those used in aminoacidcode. 
    
gencodeweights: keys are codons, values are $w_i$ for each codon

In [ ]:
aminoacidweights, gencodeweights = get_codon_weights(codon_frequencies_dic)

We pickle dump everything so we do not have to repeat the above line later. 

In [ ]:
pickle.dump( aminoacidweights,
            open( "data/Smed_transcriptome_aminoacidweights.p", "wb" ) )
pickle.dump( gencodeweights, 
            open( "data/Smed_transcriptome_gencodeweights.p", "wb" ) )
pickle.dump( aminoacidcode,
            open( "data/aminoacidcode.p", "wb" ))
pickle.dump( doubletscode,
            open( "data/doubletscode.p", "wb" ))

We reload everything with pickle because why not. 

In [ ]:
aminoacidweights = pickle.load( open( "data/Smed_transcriptome_aminoacidweights.p",
                                     "rb" ) )
gencodeweights = pickle.load( open( "data/Smed_transcriptome_gencodeweights.p", 
                                   "rb" ) )
aminoacidcode = pickle.load(open("data/aminoacidcode.p", 'rb'))
doubletscode = pickle.load(
            open( "data/doubletscode.p", "rb" ))

## We recode the luc ORFS!!!! 

Since SmedNluc2 is so short we must RNAi the whole thing. 

In [ ]:
SmedNluc2_ORF = 'ATGGTGTTTACTTTGGAAGATTTTGTTGGAGATTGGAGACAAACTGCTGGTTACAATCTGGATCAGGTACTGGAACAAGGCGGTGTTAGTTCATTATTCCAAAACCTGGGTGTGAGTGTAACTCCGATTCAGCGAATAGTGTTGTCTGGAGAAAATGGGCTGAAGATTGATATACACGTCATAATTCCATACGAAGGCTTAAGCGGTGATCAAATGGGACAAATTGAAAAAATTTTTAAAGTAGTTTACCCAGTTGACGACCATCATTTTAAAGTTATCCTTCATTACGGTACACTGGTTATAGATGGTGTAACTCCAAATATGATCGATTATTTCGGAAGACCTTACGAAGGCATAGCCGTTTTTGATGGAAAAAAGATTACAGTAACAGGTACATTGTGGAACGGAAATAAGATTATTGACGAACGTTTAATTAACCCAGATGGAAGTTTGCTCTTTAGAGTTACAATTAATGGTGTGACAGGATGGAGATTATGCGAACGGATACTCGCGTAA'

In [ ]:
SmedNluc2_protein = 'MVFTLEDFVGDWRQTAGYNLDQVLEQGGVSSLFQNLGVSVTPIQRIVLSGENGLKIDIHVIIPYEGLSGDQMGQIEKIFKVVYPVDDHHFKVILHYGTLVIDGVTPNMIDYFGRPYEGIAVFDGKKITVTGTLWNGNKIIDERLINPDGSLLFRVTINGVTGWRLCERILA*'

In [ ]:
Hluc_ORF = 'ATGGTCTTCACACTCGAAGATTTCGTTGGGGACTGGCGACAGACAGCCGGCTACAACCTGGACCAAGTCCTTGAACAGGGAGGTGTGTCCAGTTTGTTTCAGAATCTCGGGGTGTCCGTAACTCCGATCCAAAGGATTGTCCTGAGCGGTGAAAATGGGCTGAAGATCGACATCCATGTCATCATCCCGTATGAAGGTCTGAGCGGCGACCAAATGGGCCAGATCGAAAAAATTTTTAAGGTGGTGTACCCTGTGGATGATCATCACTTTAAGGTGATCCTGCACTATGGCACACTGGTAATCGACGGGGTTACGCCGAACATGATCGACTATTTCGGACGGCCGTATGAAGGCATCGCCGTGTTCGACGGCAAAAAGATCACTGTAACAGGGACCCTGTGGAACGGCAACAAAATTATCGACGAGCGCCTGATCAACCCCGACGGCTCCCTGCTGTTCCGAGTAACCATCAACGGAGTGACCGGCTGGCGGCTGTGCGAACGCATTCTGGCGTAA'

I wonder what the CAI for each ORF is?

In [ ]:
print('CAI for SMed Nuc:', get_CAI(SmedNluc2_ORF, gencodeweights))
print('CAI for Human Nuc:', get_CAI(Hluc_ORF, gencodeweights))
print('Hamming Distance vs Smed vs Human Nuc', get_hamming_dist(SmedNluc2_ORF, Hluc_ORF))

Now we can use the function get_RNAi_seq to randomly sample different recoded Luc proteins. 

The function get_RNAi_seq requires the ORF, protein sequence, an aminoacidweights and gencodeweights dictionary. We run 1000 random samples and do not enforce that every codon be different. It returns the list of tested sequences (seqs), scores ($CAI + D$/2) for each sequence, codon adaptation indices (CAIs), and Hamming distances (dists = $D$). 

In [ ]:
def get_doublest_likelihood(dna_seq, weights_dic):
    '''
    Obtains Codon Adaptation Index (CAI) for a given DNA_seq calculated using weights_dic
    CAI = (w_1*.w_i*..w_N)^(1/N) where w_i is the weight of codon i. 
    
    Inputs:
        dna_seq: ORF in form of string to evaluate CAI
        weights_dic: dictionary of CAI weights for each codon. Values are weights and keys are codons. 
    '''
    if len(dna_seq) % 3 > 0.:
        raise ValueError("Length of DNA sequence must be divisble by 3")
    ncodons = int(len(dna_seq)//3)
    score = 0. 
    for i in range(ncodons-1):
        start = i*3
        end = start + 6
        codonpair = dna_seq[start:end].upper()
        score = score+ np.log(weights_dic[codonpair])
    return score

In [ ]:
seqs, scores, cais, dists = get_RNAi_seq(SmedNluc2_ORF, SmedNluc2_protein, aminoacidweights, 
                            gencodeweights, trials = 1000,  enforce_different_codons = False, random = True)


best_seq, best_score, best_cai, best_dist = get_RNAi_seq(SmedNluc2_ORF, SmedNluc2_protein, aminoacidweights, 
                            gencodeweights, trials = 1,  enforce_different_codons = False, random = False)

best_doublet = get_doublest_likelihood(best_seq[0], doubletscode)
doublets_scores = np.array([get_doublest_likelihood(seq, doubletscode) for seq in seqs])
print(best_cai, best_dist, best_doublet)

We redo the process but enforce that every codon must be different. 

In [ ]:
seqs_diff, scores_diff, cais_diff, dists_diff = get_RNAi_seq(SmedNluc2_ORF, SmedNluc2_protein, aminoacidweights, 
                            gencodeweights, trials = 1000,  enforce_different_codons = True, random = True)

best_seq_diff, best_score_diff, best_cai_diff, best_dist_diff = get_RNAi_seq(SmedNluc2_ORF, SmedNluc2_protein, aminoacidweights, 
                            gencodeweights, trials = 1,  enforce_different_codons = True, random = False)
best_doublet_diff = get_doublest_likelihood(best_seq_diff[0], doubletscode)
doublets_scores_diff = np.array([get_doublest_likelihood(seq, doubletscode) for seq in seqs_diff])
print(best_cai_diff, best_dist_diff, best_doublet_diff)

We find the best sequences of our random simulation

In [ ]:
print(np.max(cais_diff), np.max(dists_diff))

We repeat with wiggle. 

In [ ]:
seqs_diff, scores_diff, cais_wiggle, dists_wiggle = get_RNAi_seq(SmedNluc2_ORF, SmedNluc2_protein, aminoacidweights,
                            gencodeweights, trials = 1000,  enforce_different_codons = True, random = True, wiggle = True,)

best_seq_diff, best_score_diff, best_cai_diff_wiggle, best_dist_diff_wiggle = get_RNAi_seq(SmedNluc2_ORF, SmedNluc2_protein, aminoacidweights, 
                            gencodeweights, trials = 1,  enforce_different_codons = True, random = False,  wiggle = True
                                                                            )
best_doublet_diff_wiggle = get_doublest_likelihood(best_seq_diff[0], doubletscode)
doublets_scores_wiggle = np.array([get_doublest_likelihood(seq, doubletscode) for seq in seqs_diff])
print(best_cai_diff_wiggle, best_dist_diff_wiggle, best_doublet_diff_wiggle)

In [ ]:
print(np.max(cais_wiggle), np.max(dists_wiggle))

Doublets baby

In [ ]:
seqs_doub, scores_doub, cais_doub, dists_doub = get_RNAi_seq(SmedNluc2_ORF, SmedNluc2_protein, aminoacidweights, 
                            gencodeweights, trials = 1000,  enforce_different_codons =True, random = True,
                                                            pairs = True, doubletscode = doubletscode)

best_seq_doub, best_score_doub, best_cai_doub, best_dist_doub = get_RNAi_seq(SmedNluc2_ORF, SmedNluc2_protein, aminoacidweights, 
                            gencodeweights, trials = 1,  enforce_different_codons = True, random = False,
                                                                            pairs = True, doubletscode = doubletscode,)
best_doublet_doub = get_doublest_likelihood(best_seq_doub[0], doubletscode)
doublets_scores_doub= np.array([get_doublest_likelihood(seq, doubletscode) for seq in seqs_doub])
print(best_cai_doub, best_dist_doub, best_doublet_doub)

In [ ]:
seqs_doub, scores_doub, cais_doub_wigg, dists_doub_wigg = get_RNAi_seq(SmedNluc2_ORF, SmedNluc2_protein, aminoacidweights, 
                            gencodeweights, trials = 1000,  enforce_different_codons =True, random = True, wiggle = True,
                                                            pairs = True, doubletscode = doubletscode)

best_seq_doub, best_score_doub, best_cai_doub_wiggle, best_dist_doub_wiggle = get_RNAi_seq(SmedNluc2_ORF, SmedNluc2_protein, aminoacidweights, 
                            gencodeweights, trials = 1,  enforce_different_codons = True, random = False, wiggle = True,
                                                                            pairs = True, doubletscode = doubletscode,)
best_doublet_doub = get_doublest_likelihood(best_seq_doub[0], doubletscode)
doublets_scores_doub_wigg = np.array([get_doublest_likelihood(seq, doubletscode) for seq in seqs_doub])
print(best_cai_doub_wiggle, best_dist_doub_wiggle, best_doublet_doub)

We define a function to compute ECDFs

We plot ECDFs of the CAIs.  

In [ ]:
# Make figure
p = bokeh.plotting.figure(
    frame_width=400,
    frame_height=300,
    x_axis_label='CAI',
    y_axis_label='ECDF',
    
)
cais, ecdf_cais = ecdf_vals(cais)
p.circle(cais, ecdf_cais, legend_label = 'Not all different ')

cais_diff, ecdf_cais_diff = ecdf_vals(cais_diff)
p.circle(cais_diff, ecdf_cais_diff, legend_label = 'all different', color = 'orange')

cais_wiggle, ecdf_cais_wiggle = ecdf_vals(cais_wiggle)
p.circle(cais_wiggle, ecdf_cais_wiggle, legend_label = 'all different wiggle', color = 'green')

cais_doub, ecdf_cais_doub = ecdf_vals(cais_doub)
p.circle(cais_doub, ecdf_cais_doub, legend_label = 'doublets', color = 'red')

cais_doub_wiggle, ecdf_cais_doub_wiggle = ecdf_vals(cais_doub_wigg)
p.circle(cais_doub_wiggle, ecdf_cais_doub_wiggle,
         legend_label = 'doublets wig', color = 'pink')


p.legend.location = 'bottom_right'
bokeh.io.show(p)

We plot ECDFs of the hamming distances 

In [ ]:
# Make figure
p = bokeh.plotting.figure(
    frame_width=400,
    frame_height=300,
    x_axis_label='Hamming Distance',
    y_axis_label='ECDF',
    
)
dists, ecdf_dists = ecdf_vals(dists)
p.circle(dists, ecdf_dists, legend_label = 'Not all different ')

dists_diff, ecdf_dists_diff = ecdf_vals(dists_diff)
p.circle(dists_diff, ecdf_dists_diff, legend_label = 'all different', color = 'orange')


dists_diff_wiggle, ecdf_dists_diff_wiggle = ecdf_vals(dists_wiggle)
p.circle(dists_diff_wiggle, ecdf_dists_diff_wiggle, legend_label = 'wiggle', color = 'green')

dists_doub, ecdf_dists_doub = ecdf_vals(dists_doub)
p.circle(dists_doub, ecdf_dists_doub, legend_label = 'doublets', color = 'red')

dists_doub_wiggle, ecdf_dists_doub_wiggle = ecdf_vals(dists_doub_wigg)
p.circle(dists_doub_wiggle, ecdf_dists_doub_wiggle,
         legend_label = 'doublets wig', color = 'pink')


p.legend.location = 'bottom_right'
p.x_range = bokeh.models.Range1d(.1, .6)
bokeh.io.show(p)

In [ ]:
# Make figure
p = bokeh.plotting.figure(
    frame_width=400,
    frame_height=300,
    x_axis_label='Hamming Distance',
    y_axis_label='ECDF',
    
)
dists, ecdf_dists = ecdf_vals(doublets_scores)
p.circle(dists, ecdf_dists, legend_label = 'Not all different ')

dists_diff, ecdf_dists_diff = ecdf_vals(doublets_scores_diff)
p.circle(dists_diff, ecdf_dists_diff, legend_label = 'all different', color = 'orange')


dists_diff_wiggle, ecdf_dists_diff_wiggle = ecdf_vals(doublets_scores_wiggle)
p.circle(dists_diff_wiggle, ecdf_dists_diff_wiggle, legend_label = 'wiggle', color = 'green')

dists_doub, ecdf_dists_doub = ecdf_vals(doublets_scores_doub)
p.circle(dists_doub, ecdf_dists_doub, legend_label = 'doublets', color = 'red')

dists_doub_wiggle, ecdf_dists_doub_wiggle = ecdf_vals(doublets_scores_doub_wigg)
p.circle(dists_doub_wiggle, ecdf_dists_doub_wiggle,
         legend_label = 'doublets wig', color = 'pink')


p.legend.location = 'bottom_right'

bokeh.io.show(p)